# EDA on Waste-to-Energy Projects Nationally

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

require('tidyverse')
require('DescTools')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse
Loading required package: DescTools


In [4]:
df = pd.read_csv('plants_merged_eji.csv')
df.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,GEOID_2020,COUNTY_x,StateDesc,STATEABBR,LOCATION,...,latitude,STATE,COUNTY_y,TRACT,contains_energy_source,Technology_Category,Category_Number,fossil_fuels,renewables,energy_category_numeric
0,1,1,20804,140000US01001020804,1001020804,1001020804,Autauga County,Alabama,AL,Census Tract 208.04; Autauga County; Alabama,...,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0.0
1,1,3,10706,140000US01003010706,1003010706,1003010706,Baldwin County,Alabama,AL,Census Tract 107.06; Baldwin County; Alabama,...,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0.0
2,1,3,11503,140000US01003011503,1003011503,1003011503,Baldwin County,Alabama,AL,Census Tract 115.03; Baldwin County; Alabama,...,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0.0
3,1,3,11604,140000US01003011604,1003011604,1003011604,Baldwin County,Alabama,AL,Census Tract 116.04; Baldwin County; Alabama,...,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0.0
4,1,5,950100,140000US01005950100,1005950100,1005950100,Barbour County,Alabama,AL,Census Tract 9501; Barbour County; Alabama,...,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0.0


In [5]:
df.shape

(88753, 194)

In [6]:
df['WTE'] = (df['Primary Technology'] == 'Municipal Solid Waste').astype(int)

In [7]:
df['WTE'].value_counts()

WTE
0    88702
1       51
Name: count, dtype: int64

In [9]:
df[df['WTE'] == 1]['E_POV200'].mean()

30.857773999999996

In [10]:
df[df['WTE'] == 0]['E_POV200'].mean()

30.456412145936667

In [13]:
%%R -i df
logistic <- glm(WTE ~ E_POV200 + E_AFAM, data=df, family="binomial")
summary(logistic)


Call:
glm(formula = WTE ~ E_POV200 + E_AFAM, family = "binomial", data = df)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -7.483465   0.287723 -26.009   <2e-16 ***
E_POV200     0.009416   0.008381   1.124   0.2612    
E_AFAM      -0.027998   0.012546  -2.232   0.0256 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 845.37  on 86333  degrees of freedom
Residual deviance: 838.04  on 86331  degrees of freedom
  (2419 observations deleted due to missingness)
AIC: 844.04

Number of Fisher Scoring iterations: 11



In [12]:
%%R

PseudoR2(logistic, which="McFadden")

    McFadden 
3.047385e-05 


In [14]:
%%R -i df
model <- lm(E_CANCER ~ WTE, data=df)
summary(model)


Call:
lm(formula = E_CANCER ~ WTE, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.4845 -1.9845  0.1155  1.7155 19.5155 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 7.984514   0.009124 875.139   <2e-16 ***
WTE         0.771486   0.379342   2.034    0.042 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.682 on 86433 degrees of freedom
  (2318 observations deleted due to missingness)
Multiple R-squared:  4.785e-05,	Adjusted R-squared:  3.628e-05 
F-statistic: 4.136 on 1 and 86433 DF,  p-value: 0.04198



In [18]:
%%R -i df
model <- lm(E_CANCER ~ WTE + E_POV200 + E_AFAM, data=df)
summary(model)


Call:
lm(formula = E_CANCER ~ WTE + E_POV200 + E_AFAM, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-8.7034 -1.5533  0.0714  1.4029 18.4148 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  9.7565296  0.0160866 606.500   <2e-16 ***
WTE          0.5535659  0.3340703   1.657   0.0975 .  
E_POV200    -0.0424622  0.0004966 -85.498   <2e-16 ***
E_AFAM      -0.0364653  0.0004260 -85.592   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.361 on 86244 degrees of freedom
  (2505 observations deleted due to missingness)
Multiple R-squared:  0.2177,	Adjusted R-squared:  0.2177 
F-statistic:  8001 on 3 and 86244 DF,  p-value: < 2.2e-16



In [19]:
%%R -i df
model <- lm(E_ASTHMA ~ WTE + E_POV200 + E_AFAM, data=df)
summary(model)


Call:
lm(formula = E_ASTHMA ~ WTE + E_POV200 + E_AFAM, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.2367 -0.7134 -0.0243  0.7056  6.5458 

Coefficients:
             Estimate Std. Error  t value Pr(>|t|)    
(Intercept) 9.1144408  0.0075042 1214.579   <2e-16 ***
WTE         0.0158129  0.1558395    0.101    0.919    
E_POV200    0.0442228  0.0002317  190.880   <2e-16 ***
E_AFAM      0.0133309  0.0001987   67.077   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.102 on 86244 degrees of freedom
  (2505 observations deleted due to missingness)
Multiple R-squared:  0.4105,	Adjusted R-squared:  0.4105 
F-statistic: 2.002e+04 on 3 and 86244 DF,  p-value: < 2.2e-16



In [20]:
%%R -i df
model <- lm(E_TOTCR ~ WTE + E_POV200 + E_AFAM, data=df)
summary(model)


Call:
lm(formula = E_TOTCR ~ WTE + E_POV200 + E_AFAM, data = df)

Residuals:
   Min     1Q Median     3Q    Max 
-18.55  -4.32  -3.71   5.41 375.77 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 24.299156   0.054671 444.459  < 2e-16 ***
WTE         -4.446491   1.137824  -3.908 9.32e-05 ***
E_POV200    -0.013186   0.001686  -7.823 5.23e-15 ***
E_AFAM       0.112808   0.001450  77.819  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 8.043 on 86323 degrees of freedom
  (2426 observations deleted due to missingness)
Multiple R-squared:  0.07172,	Adjusted R-squared:  0.07169 
F-statistic:  2223 on 3 and 86323 DF,  p-value: < 2.2e-16



In [22]:
%%R -i df
model <- lm(E_PM ~ WTE, data=df)
summary(model)


Call:
lm(formula = E_PM ~ WTE, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.3036 -0.3036 -0.3036 -0.1666  9.6364 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.303647   0.002646  114.77   <2e-16 ***
WTE         -0.184765   0.109303   -1.69    0.091 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7804 on 87046 degrees of freedom
  (1705 observations deleted due to missingness)
Multiple R-squared:  3.283e-05,	Adjusted R-squared:  2.134e-05 
F-statistic: 2.857 on 1 and 87046 DF,  p-value: 0.09096



In [21]:
%%R -i df
model <- lm(E_TOTCR ~ fossil_fuels + E_POV200 + E_AFAM, data=df)
summary(model)


Call:
lm(formula = E_TOTCR ~ fossil_fuels + E_POV200 + E_AFAM, data = df)

Residuals:
   Min     1Q Median     3Q    Max 
-18.60  -4.39  -3.62   5.34 375.70 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  24.376583   0.054750 445.232  < 2e-16 ***
fossil_fuels -3.313292   0.184083 -17.999  < 2e-16 ***
E_POV200     -0.013047   0.001683  -7.754 9.01e-15 ***
E_AFAM        0.112068   0.001448  77.414  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 8.029 on 86323 degrees of freedom
  (2426 observations deleted due to missingness)
Multiple R-squared:  0.07502,	Adjusted R-squared:  0.07499 
F-statistic:  2334 on 3 and 86323 DF,  p-value: < 2.2e-16

